In [3]:
import langgraph


In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langgraph.graph import MessagesState,StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
import os 
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

In [30]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] ="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [69]:
class Categorization(BaseModel):
    """Respond to the user with this"""
    summarize_content: str = Field(description="give me a summarization of the document")
    person_name: str = Field(description="Name of the person in the document")
    govertment_id: str = Field(description="Government ID of the person in the document")
    email_address: str = Field(description="Email address of the person in the document")
    type_of_document: str = Field(description=''' the document is of what type, cho0se from the folllowing
                                  credit card,
                                  savings account,
                                  driver's license,
                                  state/country identification,
                                  passport,
                                  income statements/paystubs,
                                  tax returns,
                                  Receipts''')

In [62]:
# Inherit 'messages' key from MessagesState, which is a list of chat messages
class AgentState(MessagesState):
    # Final structured response from the agent
    final_response: Categorization

In [33]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [70]:
model_with_structured_output = llm.with_structured_output(Categorization)

In [36]:
file_path="/Users/guru/appian_ai/test1.pdf"

In [41]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)


In [54]:
type(pages[0].page_content)

str

In [58]:
for page in pages:
    content= page.page_content + content

In [59]:
content

'Email address : paramaguruvh@gmail.com\nType of document :\n     Drivers licence\nNAME:PARAMAGURU\nGovermen od: 12345687\nEmail address : paramaguruvh@gmail.com\nType of document :\n     Drivers licence\n\n'

In [44]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'source': '/Users/guru/appian_ai/test1.pdf', 'page': 0}

NAME:PARAMAGURU
Govermen od: 12345687



In [39]:
cont=pages[0].page_content

In [71]:
model_with_structured_output.invoke(content)

Categorization(summarize_content='', person_name='PARAMAGURU', govertment_id='12345687', email_address='paramaguruvh@gmail.com', type_of_document="driver's license")